In [51]:
import requests
import pandas as pd
from datetime import datetime
from dateutil import parser
import pytz
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets


def get_today_game_ids():
    url = "https://statsapi.mlb.com/api/v1/schedule?sportId=1"
    resp = requests.get(url).json()
    games = resp.get('dates', [])
    if not games:
        return []
    return [game['gamePk'] for game in games[0].get('games', [])]


def get_team_lineup(game_pk, team_type):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url).json()

    team_info = resp['liveData']['boxscore']['teams'][team_type]
    team_name = team_info['team']['name']

    lineup = []
    for pid, pdata in team_info['players'].items():
        bo = pdata.get('battingOrder')
        if bo:
            try:
                bo_int = int(bo)
                if 100 <= bo_int <= 900:
                    lineup.append({
                        'Order': bo_int // 100,
                        'Name': pdata['person']['fullName'],
                        'Pos': pdata['position']['abbreviation']
                    })
            except Exception:
                continue

    lineup = sorted(lineup, key=lambda x: x['Order'])
    df = pd.DataFrame(lineup)
    if df.empty:
        df = pd.DataFrame(columns=['Name', 'Pos'])

    # Get starting pitcher name
    sp_name = "-"
    try:
        sp_id = team_info.get('startingPitcher', None)
        if sp_id:
            sp_key = f'ID{sp_id}'
            if sp_key in team_info['players']:
                sp_name = team_info['players'][sp_key]['person']['fullName']
        else:
            pitchers = team_info.get('pitchers', [])
            if pitchers:
                sp_name = team_info['players'][f'ID{pitchers[0]}']['person']['fullName']
    except Exception:
        sp_name = "-"

    return team_name, df, sp_name


def get_venue_info(game_pk):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url).json()
    try:
        venue_name = resp['gameData']['venue']['name']
        venue_id = resp['gameData']['venue']['id']
    except Exception:
        venue_name = "-"
        venue_id = "-"
    return venue_name, venue_id


def get_game_start_time(game_pk):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url).json()
    try:
        game_date_str = resp['gameData']['datetime']['dateTime']  # ISO8601 with tz
        dt_utc = parser.isoparse(game_date_str)
        est = pytz.timezone('US/Eastern')
        dt_est = dt_utc.astimezone(est)
        return dt_est.strftime('%I:%M %p EST').lstrip('0')
    except Exception:
        return "-"


def show_all_lineups(_=None):
    clear_output(wait=True)
    now_str = datetime.now().strftime('%I:%M:%S %p').lstrip('0')
    print(f"Updated at {now_str}\n")

    game_ids = get_today_game_ids()
    if not game_ids:
        print("No games today.")
        return

    for game_pk in game_ids:
        try:
            away_name, away_df, away_sp = get_team_lineup(game_pk, 'away')
            home_name, home_df, home_sp = get_team_lineup(game_pk, 'home')
            venue_name, venue_id = get_venue_info(game_pk)
            game_time_str = get_game_start_time(game_pk)

            away_df.index = list(range(1, len(away_df) + 1))
            home_df.index = list(range(1, len(home_df) + 1))

            combined = pd.DataFrame(index=range(1, 10))
            combined['Name'] = away_df['Name']
            combined['Pos'] = away_df['Pos']
            combined['Name_2'] = home_df['Name']
            combined['Pos_2'] = home_df['Pos']

            sp_row = pd.DataFrame({
                'Name': [away_sp],
                'Pos': ['SP'],
                'Name_2': [home_sp],
                'Pos_2': ['SP']
            }, index=[0])

            combined = pd.concat([sp_row, combined])
            combined.fillna("-", inplace=True)

            # Rename columns to avoid .1 and keep clear separation for styling
            combined.columns = ['Name', 'Pos', 'Name_2', 'Pos_2']

            # Display header and venue/time centered
            display(HTML(f"<div style='text-align:center; font-weight:bold; font-size:16px; margin-bottom:5px;'>{away_name} @ {home_name}</div>"))
            display(HTML(f"<div style='text-align:center; font-style:italic; margin-bottom:5px;'>{venue_name} ({venue_id})</div>"))
            display(HTML(f"<div style='text-align:center; font-style:italic; margin-bottom:10px;'>{game_time_str}</div>"))

            styled = combined.style.set_table_attributes(
                "style='margin-left:auto; margin-right:auto; border-collapse: collapse;'"
            ).set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},
                {'selector': 'td', 'props': [('text-align', 'center')]}
            ]).set_caption("Lineups and Starters")

            # Rename columns for display (no suffixes)
            combined.columns = ['Name', 'Pos', 'Name', 'Pos']

            display(styled)

        except Exception as e:
            print(f"Error loading game {game_pk}: {e}")


input_widget = widgets.Text(description="Press Enter to refresh:")
input_widget.continuous_update = False
input_widget.on_submit(show_all_lineups)

display(input_widget)

show_all_lineups()


Updated at 1:34:48 PM



,Name,Pos,Name,Pos
0,Erick Fedde,SP,Jacob deGrom,SP
1,Lars Nootbaar,LF,Josh Smith,1B
2,Masyn Winn,SS,Wyatt Langford,CF
3,Brendan Donovan,2B,Corey Seager,SS
4,Willson Contreras,1B,Josh Jung,3B
5,Iván Herrera,DH,Alejandro Osuna,LF
6,Nolan Arenado,3B,Jake Burger,DH
7,Alec Burleson,RF,Jonah Heim,C
8,Pedro Pagés,C,Marcus Semien,2B
9,Victor Scott II,CF,Sam Haggerty,RF


,Name,Pos,Name,Pos
0,Garrett Crochet,SP,Bryce Elder,SP
1,Jarren Duran,LF,Ronald Acuña Jr.,RF
2,Rafael Devers,DH,Austin Riley,3B
3,Wilyer Abreu,RF,Marcell Ozuna,DH
4,Abraham Toro,1B,Matt Olson,1B
5,Marcelo Mayer,3B,Ozzie Albies,2B
6,Trevor Story,SS,Sean Murphy,C
7,Kristian Campbell,2B,Eli White,LF
8,Connor Wong,C,Michael Harris II,CF
9,Ceddanne Rafaela,CF,Nick Allen,SS


,Name,Pos,Name,Pos
0,Adrian Houser,SP,Charlie Morton,SP
1,Chase Meidroth,SS,Jackson Holliday,2B
2,Mike Tauchman,DH,Ramón Urías,3B
3,Miguel Vargas,1B,Gunnar Henderson,SS
4,Andrew Benintendi,LF,Ryan O'Hearn,DH
5,Lenyn Sosa,2B,Coby Mayo,1B
6,Joshua Palacios,RF,Dylan Carlson,LF
7,Luis Robert Jr.,CF,Jorge Mateo,CF
8,Edgar Quero,C,Heston Kjerstad,RF
9,Josh Rojas,3B,Maverick Handley,C


,Name,Pos,Name,Pos
0,Jose Quintana,SP,Ranger Suárez,SP
1,Jackson Chourio,CF,Trea Turner,SS
2,William Contreras,C,Kyle Schwarber,DH
3,Christian Yelich,DH,Alec Bohm,1B
4,Rhys Hoskins,1B,Nick Castellanos,RF
5,Daz Cameron,LF,J.T. Realmuto,C
6,Sal Frelick,RF,Edmundo Sosa,3B
7,Caleb Durbin,3B,Weston Wilson,LF
8,Joey Ortiz,SS,Bryson Stott,2B
9,Brice Turang,2B,Johan Rojas,CF


...

,Name,Pos,Name,Pos
0,Nick Martinez,SP,Jameson Taillon,SP
1,TJ Friedl,CF,Ian Happ,LF
2,Santiago Espinal,3B,Kyle Tucker,RF
3,Elly De La Cruz,SS,Seiya Suzuki,DH
4,Gavin Lux,DH,Pete Crow-Armstrong,CF
5,Tyler Stephenson,C,Dansby Swanson,SS
6,Will Benson,LF,Michael Busch,1B
7,Spencer Steer,1B,Nico Hoerner,2B
8,Jake Fraley,RF,Matt Shaw,3B
9,Matt McLain,2B,Reese McGuire,C


,Name,Pos,Name,Pos
0,-,SP,Corbin Burnes,SP
1,-,-,Ketel Marte,DH
2,-,-,Geraldo Perdomo,SS
3,-,-,Lourdes Gurriel Jr.,LF
4,-,-,Eugenio Suárez,3B
5,-,-,Josh Naylor,1B
6,-,-,Randal Grichuk,RF
7,-,-,Ildemaro Vargas,2B
8,-,-,Tim Tawa,CF
9,-,-,Jose Herrera,C


,Name,Pos,Name,Pos
0,Chris Paddack,SP,Luis Castillo,SP
1,Trevor Larnach,RF,J.P. Crawford,SS
2,Ryan Jeffers,C,Jorge Polanco,DH
3,Matt Wallner,DH,Julio Rodríguez,CF
4,Carlos Correa,SS,Cal Raleigh,C
5,Willi Castro,LF,Randy Arozarena,LF
6,Kody Clemens,2B,Rowdy Tellez,1B
7,Ty France,1B,Leody Taveras,RF
8,Harrison Bader,CF,Cole Young,2B
9,Royce Lewis,3B,Ben Williamson,3B


,Name,Pos,Name,Pos
0,-,SP,-,SP
1,-,-,-,-
2,-,-,-,-
3,-,-,-,-
4,-,-,-,-
5,-,-,-,-
6,-,-,-,-
7,-,-,-,-
8,-,-,-,-
9,-,-,-,-


,Name,Pos,Name,Pos
0,-,SP,-,SP
1,-,-,-,-
2,-,-,-,-
3,-,-,-,-
4,-,-,-,-
5,-,-,-,-
6,-,-,-,-
7,-,-,-,-
8,-,-,-,-
9,-,-,-,-


In [63]:
import requests
import pandas as pd
from datetime import datetime
from dateutil import parser
import pytz
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets


def get_today_game_ids():
    url = "https://statsapi.mlb.com/api/v1/schedule?sportId=1"
    resp = requests.get(url).json()
    games = resp.get('dates', [])
    if not games:
        return []
    return [game['gamePk'] for game in games[0].get('games', [])]


def get_team_lineup(game_pk, team_type):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url).json()

    team_info = resp['liveData']['boxscore']['teams'][team_type]
    team_name = team_info['team']['name']

    lineup = []
    for pid, pdata in team_info['players'].items():
        bo = pdata.get('battingOrder')
        if bo:
            try:
                bo_int = int(bo)
                if 100 <= bo_int <= 900:
                    lineup.append({
                        'Order': bo_int // 100,
                        'Name': pdata['person']['fullName'],
                        'Pos': pdata['position']['abbreviation']
                    })
            except Exception:
                continue

    lineup = sorted(lineup, key=lambda x: x['Order'])
    df = pd.DataFrame(lineup)
    if df.empty:
        df = pd.DataFrame(columns=['Name', 'Pos'])

    # Get starting pitcher name
    sp_name = "-"
    try:
        sp_id = team_info.get('startingPitcher', None)
        if sp_id:
            sp_key = f'ID{sp_id}'
            if sp_key in team_info['players']:
                sp_name = team_info['players'][sp_key]['person']['fullName']
        else:
            pitchers = team_info.get('pitchers', [])
            if pitchers:
                sp_name = team_info['players'][f'ID{pitchers[0]}']['person']['fullName']
    except Exception:
        sp_name = "-"

    return team_name, df, sp_name


def get_venue_info(game_pk):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url).json()
    try:
        venue_name = resp['gameData']['venue']['name']
        venue_id = resp['gameData']['venue']['id']
    except Exception:
        venue_name = "-"
        venue_id = "-"
    return venue_name, venue_id


def get_game_start_time(game_pk):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_pk}/feed/live"
    resp = requests.get(url).json()
    try:
        game_date_str = resp['gameData']['datetime']['dateTime']  # ISO8601 with tz
        dt_utc = parser.isoparse(game_date_str)
        est = pytz.timezone('US/Eastern')
        dt_est = dt_utc.astimezone(est)
        return dt_est.strftime('%I:%M %p EST').lstrip('0')
    except Exception:
        return "-"


def show_all_lineups(_=None):
    clear_output(wait=True)
    now_str = datetime.now().strftime('%I:%M:%S %p').lstrip('0')
    print(f"Updated at {now_str}\n")

    game_ids = get_today_game_ids()
    if not game_ids:
        print("No games today.")
        return

    all_game_html = []  # To hold each game’s HTML block

    for game_pk in game_ids:
        try:
            away_name, away_df, away_sp = get_team_lineup(game_pk, 'away')
            home_name, home_df, home_sp = get_team_lineup(game_pk, 'home')
            venue_name, venue_id = get_venue_info(game_pk)
            game_time_str = get_game_start_time(game_pk)

            away_df.index = list(range(1, len(away_df) + 1))
            home_df.index = list(range(1, len(home_df) + 1))

            combined = pd.DataFrame(index=range(1, 10))
            combined['Name'] = away_df['Name']
            combined['Pos'] = away_df['Pos']
            combined['Name'] = combined['Name']  # to avoid pandas warning
            combined['Pos'] = combined['Pos']
            combined['Name_2'] = home_df['Name']
            combined['Pos_2'] = home_df['Pos']

            sp_row = pd.DataFrame({
                'Name': [away_sp],
                'Pos': ['SP'],
                'Name_2': [home_sp],
                'Pos_2': ['SP']
            }, index=[0])

            combined = pd.concat([sp_row, combined])
            combined.fillna("-", inplace=True)

            # Rename columns to remove suffixes for display
            combined.columns = ['Name', 'Pos', 'Name', 'Pos']

            # Create styled table and convert to HTML
            styled = combined.style.set_table_attributes(
                "style='margin-left:auto; margin-right:auto; border-collapse: collapse;'"
            ).set_table_styles([
                {'selector': 'th', 'props': [('text-align', 'center')]},
                {'selector': 'td', 'props': [('text-align', 'center')]}
            ]).set_caption("Lineups and Starters")

            table_html = styled.to_html()

            # Build the full block HTML for this game with header, venue, time, and table
            game_html = f"""
            <div style="display:inline-block; vertical-align:top; width:30%; margin-right: 20px; box-sizing: border-box;">
                <div style="text-align:center; font-weight:bold; font-size:16px; margin-bottom:5px;">{away_name} @ {home_name}</div>
                <div style="text-align:center; font-style:italic; margin-bottom:5px;">{venue_name} ({venue_id})</div>
                <div style="text-align:center; font-style:italic; margin-bottom:10px;">{game_time_str}</div>
                {table_html}
            </div>
            """
            all_game_html.append(game_html)

        except Exception as e:
            print(f"Error loading game {game_pk}: {e}")

    # Join all games blocks side-by-side
    display(HTML("<div style='width:100%; display:flex; flex-wrap: wrap; justify-content: center;'>" +
                 "".join(all_game_html) +
                 "</div>"))


input_widget = widgets.Text(description="Press Enter to refresh:")
input_widget.continuous_update = False
input_widget.on_submit(show_all_lineups)

display(input_widget)

show_all_lineups()


Updated at 2:07:56 PM



,Name,Pos,Name,Pos
0,Erick Fedde,SP,Jacob deGrom,SP
1,Lars Nootbaar,LF,Josh Smith,1B
2,Masyn Winn,SS,Wyatt Langford,CF
3,Brendan Donovan,2B,Corey Seager,SS
4,Willson Contreras,1B,Josh Jung,3B
5,Iván Herrera,DH,Alejandro Osuna,LF
6,Nolan Arenado,3B,Jake Burger,DH
7,Alec Burleson,RF,Jonah Heim,C
8,Pedro Pagés,C,Marcus Semien,2B
9,Victor Scott II,CF,Sam Haggerty,RF
